In [1]:
import timesynth as ts
import os
import json
import pickle as pkl
import copy
import matplotlib.pyplot as plt
import time
import pandas as pd

In [2]:
dataDir = './data'  # output all the generated data into this directory

In [3]:
os.popen("mkdir " + dataDir)

## 正弦波 + 白噪声

In [4]:
sinDataDir = dataDir + '/Sinusoidal'

In [5]:
os.popen("mkdir " + sinDataDir)

### base data

In [6]:
baseConfigDict = {
    "signal_type": "Sinusoidal",  # type of the signal
    "stop_time": 20,  # total time
    "num_points": 500,  # number of sample points
    "keep_percentage": 50,  # 
    "frequency": 0.25,  # frequency of the sine wave
    "amplitude": 1,  # amplitude of the sine wave
    "noise": "white_noise",  # white_noise or red_noise
    "std": 0.3,  # std of the white noise or red noise
    "tau": 0  # useful only when "noise" is "red_noise"
}

In [7]:
def generateSinusoidSeries(configDict, outputDir, numSeries):
    """
    this function generates the time series according to the dictionary configData, 
        and output it to the directory outputDir
    
    :param configDict: a dictionary in the same format as the baseConfigDict above, specifies the configurations
    :param outputDir: output the generated data to this directory
    :param numSeries: how many series to generate
    """
    
    # output the json object
    configJSON = json.dumps(configDict)
    while not os.path.exists(outputDir) == True:
        time.sleep(0.01)  # 注解：见下面
    JSONFile = open(os.path.join(outputDir,"config.json"), "w")
    JSONFile.write(configJSON)
    JSONFile.close()
    
    for ii in range(numSeries):
        # generate data, almost the same as the code shown in README.md
        time_sampler = ts.TimeSampler(stop_time=configDict["stop_time"])
        irregular_time_samples = time_sampler.sample_irregular_time(num_points=configDict["num_points"], 
                                                                    keep_percentage=configDict["keep_percentage"])
        sinusoid = ts.signals.Sinusoidal(frequency=configDict["frequency"], amplitude=configDict["amplitude"])
        if configDict["noise"] == "white_noise":
            noise_generator = ts.noise.GaussianNoise(std=configDict["std"])
        elif configDict["noise"] == "red_noise":
            noise_generator = ts.noise.RedNoise(std=configDict["std"], tau=configDict["tau"])
        else:
            raise Exception("Unknown type of noise!")
        timeseries = ts.TimeSeries(sinusoid, noise_generator=noise_generator)
        samples, signals, errors = timeseries.sample(irregular_time_samples)
        # plot the generated series to see whether I got it right
        ## plt.plot(irregular_time_samples, samples, marker='o')
        
        # output the data (including the time data and the magnitude data) to the given directory
        outData = pd.DataFrame([irregular_time_samples, samples], index=["mjd", "mag"])
        pklFile = open(os.path.join(outputDir,"%d.pkl" % ii), "wb")
        pkl.dump(outData, pklFile)
        pklFile.close()
        

对`sample_irregular_time`函数的说明：有三个关键字参数`num_points, resolution, keep_percentage`，其中`num_points`和`resolution`的作用相同（同时给出这两个参数时，只有一个有效）。

首先将`TimeSampler`类的对象中的`start_time`和`stop_time`之间的区间按照`num_points`或者`resolution`等间隔取点，再从中随机选取keep_percentage%比例的点，最后对这些点施加扰动，得到所需的时间序列

In [8]:
NUM_SERIES = 500  # how many series to generate

In [9]:
currentDataDir = sinDataDir + '/base'
os.popen("mkdir "+currentDataDir)
generateSinusoidSeries(baseConfigDict, currentDataDir, NUM_SERIES)

关于`sleep(0.01)`那一句的说明：如果不加上这一句，会发生“找不到/base这个目录”的错误，疑似是因为`os.popen("mkdir "+currentDataDir)`这一句只是向操作系统发送了一条创建/base目录的指令，而此程序没有被阻塞，因此在操作系统创建/base这个目录之前，就调用了`JSONFile = open(os.path.join(outputDir,"config.json"), "w")`这一句，造成找不到/base目录的错误。

在这里加一个输入语句`x = input()`可以起到相同的效果，但是比较麻烦，需要手动按一下回车。

### 改变正弦波的频率

In [10]:
freq = [0.1, 0.5, 0.8, 1]  # the frequencies to choose
for f in freq:
    configDict = copy.deepcopy(baseConfigDict)
    configDict['frequency'] = f
    currentDataDir = sinDataDir + '/freq=' + str(f)
    os.popen("mkdir "+currentDataDir)
    generateSinusoidSeries(configDict, currentDataDir, NUM_SERIES)

### 改变采样频率
实际上是改变采样点数num_points（也可以改resolution，不过既然示例里面用的是采样点数就改采样点数吧）

In [11]:
num_points = [1000, 1500, 2000]
for n in num_points:
    configDict = copy.deepcopy(baseConfigDict)
    configDict['num_points'] = n
    currentDataDir = sinDataDir + '/num_points=' + str(n)
    os.popen("mkdir "+currentDataDir)
    generateSinusoidSeries(configDict, currentDataDir, NUM_SERIES)

### 改变正弦波的幅值

In [12]:
amplitude = [0.5, 2, 5]
for A in amplitude:
    configDict = copy.deepcopy(baseConfigDict)
    configDict['amplitude'] = A
    currentDataDir = sinDataDir + '/amplitude=' + str(A)
    os.popen("mkdir "+currentDataDir)
    generateSinusoidSeries(configDict, currentDataDir, NUM_SERIES)

### 改变噪声的特性

#### 白噪声（高斯噪声）

In [13]:
sigma = [0.05, 0.1, 0.5]
for s in sigma:
    configDict = copy.deepcopy(baseConfigDict)
    configDict['std'] = s
    currentDataDir = sinDataDir + '/white_noise,std=' + str(s)
    os.popen("mkdir "+currentDataDir)
    generateSinusoidSeries(configDict, currentDataDir, NUM_SERIES)

#### red noise

In [14]:
parameters = [(0.5, 0.8), (0.5, 0.4), (0.5, 1), (0.2, 0.8), (0.1, 0.8)]  # （std, tau)的组合
for sigma, tau in parameters:
    configDict = copy.deepcopy(baseConfigDict)
    configDict['noise'] = 'red_noise'
    configDict['std'] = sigma
    configDict['tau'] = tau
    currentDataDir = sinDataDir + '/red_noise,std=' + str(sigma) + ',tau=' + str(tau)
    os.popen("mkdir "+currentDataDir)
    generateSinusoidSeries(configDict, currentDataDir, 20)

## GP

### base data

In [15]:
GPDataDir = dataDir + '/GP'
os.popen("mkdir "+GPDataDir)

In [16]:
baseConfigDict = {
    "signal_type": "GP",  # type of the signal
    "stop_time": 20,  # total time
    "num_points": 500,  # number of sample points
    "keep_percentage": 50,  # 
    "variance": 1.,  # variance of a single normal distribution random variable
    "kernel": ""  # kernal function of the GP
    # 与kernel有关的一些常数，还需要再加入此dictionary中，例如当kernel取"Matern"时，关键字中还要加上"nu"，键值为nu的值
}

In [17]:
def generateGPSeries(configDict, outputDir, numSeries):
    """
    this function generates the time series according to the dictionary configData, 
        and output it to the directory outputDir
    
    :param configDict: a dictionary in the same format as the baseConfigDict above, specifies the configurations
    :param outputDir: output the generated data to this directory
    :param numSeries: how many series to generate
    """
    
    # output the json object
    configJSON = json.dumps(configDict)
    while os.path.exists(outputDir) is not True:
        time.sleep(0.01)
    JSONFile = open(os.path.join(outputDir,"config.json"), "w")
    JSONFile.write(configJSON)
    JSONFile.close()
    
    for ii in range(numSeries):
        # generate data, almost the same as the code shown in TimeSynthExamples.ipynb
        time_sampler = ts.TimeSampler(stop_time=configDict["stop_time"])
        irregular_time_samples = time_sampler.sample_irregular_time(num_points=configDict["num_points"], 
                                                                    keep_percentage=configDict["keep_percentage"])
        if configDict['kernel'] == 'Constant':
            gp = ts.signals.GaussianProcess(kernel='Constant', variance=configDict['variance'])
        elif configDict['kernel'] == 'Exponential':
            gp = ts.signals.GaussianProcess(kernel='Exponential', variance=configDict['variance'],
                                            gamma=configDict['gamma'])
        elif configDict['kernel'] == 'Matern':
            gp = ts.signals.GaussianProcess(kernel='Matern', variance=configDict['variance'],
                                            nu=configDict['nu'])
        elif configDict['kernel'] == 'RQ':
            gp = ts.signals.GaussianProcess(kernel='RQ', variance=configDict['variance'],
                                            alpha=configDict['alpha'])
        else: 
            raise Exception ("currently not implemented")
        gp_series = ts.TimeSeries(signal_generator=gp)
        samples = gp_series.sample(irregular_time_samples)[0]
        # plot the generated series to see whether I got it right
        ## plt.plot(irregular_time_samples, samples, marker='o')
        
        # output the data (including the time data and the magnitude data) to the given directory
        outData = pd.DataFrame([irregular_time_samples, samples], index=["mjd", "mag"])
        pklFile = open(os.path.join(outputDir,"%d.pkl" % ii), "wb")
        pkl.dump(outData, pklFile)
        pklFile.close()
        

In [18]:
sigma = [0.1, 0.5, 1., 2., 5.]

### kernel fun = Constant

注：下面这个单元格不要运行！Constant的kernel function得到的结果是是所有正态随机变量之间的相关系数均为1，所以画出来的图像是一条直线。

In [19]:
# for s in sigma:
#     configDict = copy.deepcopy(baseConfigDict)
#     configDict['kernel'] = 'Constant'
#     configDict['variance'] = s ** 2
#     currentDataDir = GPDataDir + '/Constant,var=' + str(s ** 2)
#     os.popen("mkdir "+currentDataDir)
#     generateGPSeries(configDict, currentDataDir, NUM_SERIES)

### ker fun = Exp

In [20]:
gamma = [0.1, 0.5, 1]
for s in sigma:
    for g in gamma:
        configDict = copy.deepcopy(baseConfigDict)
        configDict['kernel'] = 'Exponential'
        configDict['variance'] = s ** 2
        configDict['gamma'] = g
        currentDataDir = GPDataDir + '/Exponential,var=' + str(s ** 2) + ',gamma=' + str(g)
        os.popen("mkdir "+currentDataDir)
        generateGPSeries(configDict, currentDataDir, NUM_SERIES)

### ker fun = Matern

In [21]:
nu = [3./2]
for s in sigma:
    for v in nu:
        configDict = copy.deepcopy(baseConfigDict)
        configDict['kernel'] = 'Matern'
        configDict['variance'] = s ** 2
        configDict['nu'] = v
        currentDataDir = GPDataDir + '/Matern,var=' + str(s ** 2) + ',nu=' + str(v)
        os.popen("mkdir "+currentDataDir)
        generateGPSeries(configDict, currentDataDir, NUM_SERIES)

### ker fun = RQ

In [22]:
alpha = [0.5, 1, 2]
for s in sigma:
    for a in alpha:
        configDict = copy.deepcopy(baseConfigDict)
        configDict['kernel'] = 'RQ'
        configDict['variance'] = s ** 2
        configDict['alpha'] = a
        currentDataDir = GPDataDir + '/RQ,var=' + str(s ** 2) + ',alpha=' + str(v)
        os.popen("mkdir "+currentDataDir)
        generateGPSeries(configDict, currentDataDir, NUM_SERIES)

### 改变采样频率

In [23]:
num_points = [1000, 1500, 2000]
for n in num_points:
    configDict = copy.deepcopy(baseConfigDict)
    configDict['num_points'] = n
    configDict['kernel'] = 'Exponential'
    configDict['variance'] = 1.
    configDict['gamma'] = 1
    currentDataDir = GPDataDir + '/num_points=' + str(n)
    os.popen("mkdir "+currentDataDir)
    generateGPSeries(configDict, currentDataDir, NUM_SERIES)

## CAR

In [29]:
CARDataDir = dataDir + '/CAR'
os.popen("mkdir "+CARDataDir)

In [30]:
baseConfigDict = {
    "signal_type": "CAR",  # type of the signal
    "stop_time": 20,  # total time
    "num_points": 500,  # number of sample points
    "keep_percentage": 50,  # 
    "ar_param": 0.9,  # parameter of the AR process
    "sigma": 1  # std of the signal 
}

In [31]:
def generateCARSeries(configDict, outputDir, numSeries):
    """
    this function generates the time series according to the dictionary configData, 
        and output it to the directory outputDir
    
    :param configDict: a dictionary in the same format as the baseConfigDict above, specifies the configurations
    :param outputDir: output the generated data to this directory
    :param numSeries: how many series to generate
    """
    
    # output the json object
    configJSON = json.dumps(configDict)
    while os.path.exists(outputDir) is not True:
        time.sleep(0.01)
    JSONFile = open(os.path.join(outputDir,"config.json"), "w")
    JSONFile.write(configJSON)
    JSONFile.close()
    
    for ii in range(numSeries):
        # generate data, almost the same as the code shown in TimeSynthExamples.ipynb
        time_sampler = ts.TimeSampler(stop_time=configDict["stop_time"])
        irregular_time_samples = time_sampler.sample_irregular_time(num_points=configDict["num_points"], 
                                                                    keep_percentage=configDict["keep_percentage"])
        car = ts.signals.CAR(ar_param=configDict['ar_param'], sigma=configDict['sigma'])
        car_series = ts.TimeSeries(signal_generator=car)
        samples = car_series.sample(irregular_time_samples)[0]
        # plot the generated series to see whether I got it right
        ## plt.plot(irregular_time_samples, samples, marker='o')
        
        # output the data (including the time data and the magnitude data) to the given directory
        outData = pd.DataFrame([irregular_time_samples, samples], index=["mjd", "mag"])
        pklFile = open(os.path.join(outputDir,"%d.pkl" % ii), "wb")
        pkl.dump(outData, pklFile)
        pklFile.close()
        

### 改变 sigma 和 ar_param

In [32]:
sigma = [1, 0.5, 0.1]
ar_param = [0.9, 0.5, 0.3]
for s in sigma:
    for p in ar_param:
        configDict = copy.deepcopy(baseConfigDict)
        configDict['ar_param'] = p
        configDict['sigma'] = s
        currentDataDir = CARDataDir + '/ar_param=' + str(p) + ',sigma=' + str(s)
        os.popen("mkdir "+currentDataDir)
        generateCARSeries(configDict, currentDataDir, NUM_SERIES)

### 改变采样频率

In [33]:
num_points = [1000, 1500, 2000]
for n in num_points:
    configDict = copy.deepcopy(baseConfigDict)
    configDict['num_points'] = n
    currentDataDir = CARDataDir + '/num_points=' + str(n)
    os.popen("mkdir "+currentDataDir)
    generateCARSeries(configDict, currentDataDir, NUM_SERIES)

----------
上面产生的数据横竖方向反了，需要reverse一下

In [2]:
for root, dirs, files in os.walk("./data"):
    for fileName in files:
        if os.path.splitext(fileName)[1] == ".pkl":
            pklFile = open(os.path.join(root, fileName), "rb")
            data = pkl.load(pklFile)
            pklFile.close()
            data = pd.DataFrame(data.values.T, index = data.columns, columns = data.index)
            pklFile = open(os.path.join(root, fileName), "wb")
            pkl.dump(data, pklFile)
            pklFile.close()